In [1]:
import qiskit
from qiskit import QuantumCircuit as QiskitCircuit
from qiskit.quantum_info import Operator as QiskitOperator
import cirq
from cirq import Circuit as CirqCircuit
import braket
from braket.circuits import Circuit as BraketCircuit
from braket.circuits.unitary_calculation import calculate_unitary
#from qbraid.transpiler.transpiler import qbraid_wrapper
from qbraid.transpiler2.transpiler import qbraid_wrapper
import numpy as np
from typing import List

### Shared gate circuit functions and unitary helper function

In [2]:
def qiskit_shared_gates_circuit():
    """Returns qiskit `QuantumCircuit` for qBraid `TestSharedGates`."""
    
    qiskit_circuit = QiskitCircuit(4)

    qiskit_circuit.h([0, 1, 2, 3])
    qiskit_circuit.x([0, 1])
    qiskit_circuit.y(2)
    qiskit_circuit.z(3)
    qiskit_circuit.s(0)
    qiskit_circuit.sdg(1)
    qiskit_circuit.t(2)
    qiskit_circuit.tdg(3)
    qiskit_circuit.rx(np.pi/4, 0)
    qiskit_circuit.ry(np.pi/2, 1)
    qiskit_circuit.rz(3*np.pi/4, 2)
    qiskit_circuit.p(np.pi/8, 3)
    qiskit_circuit.sx(0)
    qiskit_circuit.sxdg(1)
    qiskit_circuit.iswap(2, 3)
    qiskit_circuit.swap([0, 1], [2, 3])
    qiskit_circuit.cx(0, 1)
    qiskit_circuit.cp(np.pi/4, 2, 3)

    return qiskit_circuit

In [3]:
def cirq_shared_gates_circuit(rev_qubits=False):
    """Returns cirq `Circuit` for qBraid `TestSharedGates` 
    rev_qubits=True reverses ordering of qubits."""
    
    cirq_circuit = CirqCircuit()
    qubits = [cirq.LineQubit(i) for i in range(4)]
    q3, q2, q1, q0 = qubits if rev_qubits else list(reversed(qubits))
        
    cirq_gates = [
        cirq.H(q0),
        cirq.H(q1),
        cirq.H(q2),
        cirq.H(q3),
        cirq.X(q0),
        cirq.X(q1),
        cirq.Y(q2),
        cirq.Z(q3),
        cirq.S(q0),
        cirq.ZPowGate(exponent=-0.5)(q1),
        cirq.T(q2),
        cirq.ZPowGate(exponent=-0.25)(q3),
        cirq.Rx(rads=np.pi/4)(q0),
        cirq.Ry(rads=np.pi/2)(q1),
        cirq.Rz(rads=3*np.pi/4)(q2),
        cirq.ZPowGate(exponent=1/8)(q3),
        cirq.XPowGate(exponent=0.5)(q0),
        cirq.XPowGate(exponent=-0.5)(q1),
        cirq.ISWAP(q2, q3),
        cirq.SWAP(q0, q2),
        cirq.SWAP(q1, q3),
        cirq.CNOT(q0, q1),
        cirq.CZPowGate(exponent=0.25)(q2, q3),
    ]
    
    for gate in cirq_gates:
        cirq_circuit.append(gate)
    
    return cirq_circuit 

In [4]:
def braket_shared_gates_circuit():
    """Returns braket `Circuit` for qBraid `TestSharedGates`."""
    
    braket_circuit = BraketCircuit()

    braket_circuit.h(0)
    braket_circuit.h(1)
    braket_circuit.h(2)
    braket_circuit.h(3)
    braket_circuit.x(0)
    braket_circuit.x(1)
    braket_circuit.y(2)
    braket_circuit.z(3)
    braket_circuit.s(0)
    braket_circuit.si(1)
    braket_circuit.t(2)
    braket_circuit.ti(3)
    braket_circuit.x(0)
    braket_circuit.x(1)
    braket_circuit.y(2)
    braket_circuit.z(3)
    braket_circuit.rx(0, np.pi / 4)
    braket_circuit.ry(1, np.pi / 2)
    braket_circuit.rz(2, 3 * np.pi / 4)
    braket_circuit.phaseshift(3, np.pi / 8)
    braket_circuit.v(0)
    braket_circuit.vi(1)
    braket_circuit.iswap(2, 3)
    braket_circuit.swap(0, 2)
    braket_circuit.swap(1, 3)
    braket_circuit.cnot(0, 1)
    braket_circuit.cphaseshift(2, 3, np.pi / 4)
    
    return braket_circuit

In [5]:
def to_unitary(circuit):
    """Calculate unitary of a braket, cirq, or qiskit circuit.
    Args:
        circuit (braket, cirq, or qiskit Circuit): The circuit object for which 
            the unitary matrix will be calculated. 
    Returns:
        numpy.ndarray: A numpy array representing the `circuit` as a unitary
    """
    if isinstance(circuit, BraketCircuit):
        return calculate_unitary(circuit.qubit_count, circuit.instructions)
    elif isinstance(circuit, CirqCircuit):
        return circuit.unitary()
    elif isinstance(circuit, QiskitCircuit):
        return QiskitOperator(circuit).data
    else:
        raise TypeError(f"to_unitary calculation not supported for type {type(circuit)}")

### Define circuits and unitaries

In [6]:
braket_circuit = braket_shared_gates_circuit()
cirq_circuit = cirq_shared_gates_circuit()
cirq_rev_circuit = cirq_shared_gates_circuit(rev_qubits=True)
qiskit_circuit = qiskit_shared_gates_circuit()

In [7]:
braket_unitary = to_unitary(braket_circuit)
cirq_unitary = to_unitary(cirq_circuit)
cirq_rev_unitary = to_unitary(cirq_rev_circuit)
qiskit_unitary = to_unitary(qiskit_circuit)

In [8]:
qbraid_circuit_braket = qbraid_wrapper(braket_circuit)
qbraid_circuit_cirq = qbraid_wrapper(cirq_circuit)
qbraid_circuit_cirq_rev = qbraid_wrapper(cirq_rev_circuit)
qbraid_circuit_qiskit = qbraid_wrapper(qiskit_circuit)

In [9]:
def test_transpile(qbraid_circuit, target_package, target_unitary):
    transpiled_circuit = qbraid_circuit.transpile(target_package)
    print("transpiled circuit\n", transpiled_circuit)
    transpiled_unitary = to_unitary(transpiled_circuit)
    success = np.allclose(transpiled_unitary, target_unitary)
    return "target = transpiled: " + str(success)

In [10]:
np.allclose(cirq_rev_unitary, qiskit_unitary, braket_unitary)

True

In [11]:
# braket to cirq
print("original circuit\n", braket_circuit, "\n")
print("target circuit\n", cirq_circuit, "\n")
test_transpile(qbraid_circuit_braket, "cirq", cirq_unitary)

original circuit
 T  : |0|1|2 |3|     4      |  5  |    6    |     7      |
                                                         
q0 : -H-X-S--X-Rx(0.785)----V-----SWAP------C------------
                                  |         |            
q1 : -H-X-Si-X-Ry(1.57)-----Vi----|----SWAP-X------------
                                  |    |                 
q2 : -H-Y-T--Y-Rz(2.36)-----ISWAP-SWAP-|----C------------
                            |          |    |            
q3 : -H-Z-Ti-Z-PHASE(0.393)-ISWAP------SWAP-PHASE(0.785)-

T  : |0|1|2 |3|     4      |  5  |    6    |     7      | 

target circuit
                                           ┌──┐
0: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                           │     │
1: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           ││
2: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@────────
                                 │          │    │
3: ───H───Z

'target = transpiled: False'

In [12]:
# braket to qiskit
print("original circuit\n", braket_circuit, "\n")
print("target circuit\n", qiskit_circuit, "\n")
test_transpile(qbraid_circuit_braket, "qiskit", qiskit_unitary)

original circuit
 T  : |0|1|2 |3|     4      |  5  |    6    |     7      |
                                                         
q0 : -H-X-S--X-Rx(0.785)----V-----SWAP------C------------
                                  |         |            
q1 : -H-X-Si-X-Ry(1.57)-----Vi----|----SWAP-X------------
                                  |    |                 
q2 : -H-Y-T--Y-Rz(2.36)-----ISWAP-SWAP-|----C------------
                            |          |    |            
q3 : -H-Z-Ti-Z-PHASE(0.393)-ISWAP------SWAP-PHASE(0.785)-

T  : |0|1|2 |3|     4      |  5  |    6    |     7      | 

target circuit
      ┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└

'target = transpiled: False'

In [13]:
# qiskit to braket
print("original circuit\n", qiskit_circuit, "\n")
print("target circuit\n", braket_circuit, "\n")
test_transpile(qbraid_circuit_qiskit, "braket", braket_unitary)

original circuit
      ┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  ISWAP │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ TDG ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘                

target circuit
 T  : |0|1|2 |3|     4      |  5  |    6    |     7      |
                                                         
q0 : -H-X-S--X-Rx(0.785)----V-----SWAP------C------------
                                  |         |            
q1 : -H-X-Si-X-Ry(1.57)-----Vi----|----SWAP-X------------
                                  |    |                 
q2 : -H-Y-T--Y-Rz(2.36)-----ISWAP-SWAP-|----C------------
                  

'target = transpiled: False'

In [14]:
# qiskit to cirq
print("original circuit\n", qiskit_circuit, "\n")
print("target circuit\n", cirq_circuit, "\n")
test_transpile(qbraid_circuit_qiskit, "cirq", cirq_unitary)

original circuit
      ┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  ISWAP │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ TDG ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘                

target circuit
                                           ┌──┐
0: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                           │     │
1: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           ││
2: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@────────
                                 │          │    │
3: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@^0

'target = transpiled: True'

In [15]:
# cirq to braket
print("original circuit\n", cirq_circuit, "\n")
print("target circuit\n", braket_circuit, "\n")
test_transpile(qbraid_circuit_cirq, "braket", braket_unitary) # Error

original circuit
                                           ┌──┐
0: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                           │     │
1: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           ││
2: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@────────
                                 │          │    │
3: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@^0.25───
                                          └──┘ 

target circuit
 T  : |0|1|2 |3|     4      |  5  |    6    |     7      |
                                                         
q0 : -H-X-S--X-Rx(0.785)----V-----SWAP------C------------
                                  |         |            
q1 : -H-X-Si-X-Ry(1.57)-----Vi----|----SWAP-X------------
                                  |    |                 
q2 : -H-Y-T--Y-Rz(2.36)-----ISWAP-SWAP-|----C------------
                            |          |    |            
q3 : -H-Z-Ti-Z-PHASE(0

'target = transpiled: False'

In [16]:
# cirq to qiskit
print("original circuit\n", cirq_circuit, "\n")
print("target circuit\n", qiskit_circuit, "\n")
test_transpile(qbraid_circuit_cirq, "qiskit", qiskit_unitary)

original circuit
                                           ┌──┐
0: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                           │     │
1: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           ││
2: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@────────
                                 │          │    │
3: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@^0.25───
                                          └──┘ 

target circuit
      ┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  ISWAP │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ TDG ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └

'target = transpiled: True'